In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json, operator
%matplotlib inline

###Class Distribution

#### Calculate fraction of documents in each class

In [5]:
#Training label
train_label = open('/home/sadat/Downloads/HW2_210/20news-bydate/matlab/train.label')

#pi is the fraction of each class
pi = {}

#Set a class index for each document as key
for i in range(1,21):
    pi[i] = 0
    
#Extract values from training labels
lines = train_label.readlines()

#Get total number of documents
total = len(lines)

#Count the occurence of each class
for line in lines:
    val = int(line.split()[0])
    pi[val] += 1

#Divide the count of each class by total documents 
for key in pi:
    pi[key] /= total
    
print('Created dictionary with', len(pi), 'classes')

Created dictionary with 20 classes


###Probability Distribution over V

####Average count of each word per class

In [6]:
#Training data
train_data = open('/home/sadat/Downloads/HW2_210/20news-bydate/matlab/train.data')
df = pd.read_csv(train_data, delimiter=' ', names=['docIdx', 'wordIdx', 'count'])

#Training label
label = []
train_label = open('/home/sadat/Downloads/HW2_210/20news-bydate/matlab/train.label')
lines = train_label.readlines()
for line in lines:
    label.append(int(line.split()[0]))
    
#Increase label length to match docIdx
docIdx = df['docIdx'].values
i = 0
new_label = []
for index in range(len(docIdx)-1):
    new_label.append(label[i])
    if docIdx[index] != docIdx[index+1]:
        i += 1
new_label.append(label[i]) #for-loop ignores last value

#Add label column
df['label'] = new_label

df.head()

,docIdx,wordIdx,count,label
0,1,1,4,1
1,1,2,2,1
2,1,3,10,1
3,1,4,4,1
4,1,5,2,1


####Inverse Document Frequency

Since the calculation using all 61188 words it taking a long time to run, we will pick the top 100 words using IDF.

In [28]:
idf = {}
i = 0
totDoc = len(df['docIdx'].unique())
for wordIdx in range(1,61189):
    i += 1
    print(i,end='\r')
    val = len(df[df['wordIdx'] == wordIdx])
    idf[wordIdx] = np.log(val/totDoc)

print('TF-IDF complete!')

TF-IDF complete!


In [45]:
#Sort idf dictionary in descending order
sorted_idf = sorted(idf.items(), key=operator.itemgetter(1), reverse=True)

#Pick top 1000 words
sorted_idf = sorted_idf[:100]

#Store top words in imp_words list
imp_words = []
for word in sorted_idf:
    imp_words.append(word[0])

####Average count of each word per class

In [ ]:
P = {}
i = 0

for label in range(1,21):
    wordList = []
    i += 1
    print(i,end='\r')
    for wordIdx in imp_words:
        count = 0
    
        #Count the word frequency in class
        cond1 = (df['label']==label) & (df['wordIdx']==wordIdx)
        count = np.nansum(df[cond1]['count'])
        
        #Count the total words in class
        cond2 = (df['label']==label)
        totDoc = np.nansum(df[cond2]['count'])
        
        #Append probability to wordList with Laplace Smoothing
        wordList.append((count+1)/(totDoc+2))
    
    P[label]=wordList

###Final Classifier

####Combining probability distribution of P with fraction of documents belonging to each class

In [48]:
classifier = {}
for label in range(1,21):
    classifier[label] = [pi[label]*i for i in P[label]]

In [56]:
#Store Classifier on JSON file
f = open('classifier.json', 'a')
json.dump(classifier, f)
f.close()